# 1) Mount drive, unzip data, clone repo, install packages

## 1.1) Define paths
Main Google Drive root: `/workspace/GoogleDrive`

In [ ]:
import os
from ipywidgets import FileUpload

# Define Google Drive related paths
drive_root = "/workspace/GoogleDrive"
!mkdir -p "$drive_root"
!mkdir -p "$drive_root/Models"
!mkdir -p "$drive_root/Datasets"
!mkdir -p "$drive_root/GitHub Keys"

# Upload ssh keys
is_first_run = not os.path.exists(f'{drive_root}/GitHub Keys/config') or not os.path.exists(f'{drive_root}/GitHub Keys/id_rsa') or not os.path.exists(f'{drive_root}/client_secrets.json')
if is_first_run:
    #   - config
    with open(f'{drive_root}/GitHub Keys/config', 'w') as fp:
        fp.writelines(['Host github.com\n', '    Hostname github.com\n', f'    IdentityFile "{drive_root}/GitHub Keys/id_rsa"\n', '    IdentitiesOnly yes\n'])
    #   - id_rsa.pub
    upload = FileUpload(multiple=True)
    display(upload)

In [ ]:
if is_first_run:
    with open(f'{drive_root}/GitHub Keys/id_rsa', 'wb') as i:
        i.write(upload.value['id_rsa']['content'])
    !chmod 600 "$drive_root/GitHub Keys/id_rsa"
    with open(f'{drive_root}/GitHub Keys/id_rsa.pub', 'wb') as i:
        i.write(upload.value['id_rsa.pub']['content'])
    !chmod 600 "$drive_root/GitHub Keys/id_rsa.pub"

    # client_secrets.json
    with open(f'{drive_root}/client_secrets.json', 'wb') as i:
        i.write(upload.value['client_secrets.json']['content'])
    
    # kaggle.json
    with open(f'/workspace/kaggle.json', 'wb') as i:
        i.write(upload.value['kaggle.json']['content'])
    !mkdir -p /root/.kaggle
    !cp -rf /workspace/kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json

## 1.2) Clone GitHub repo
Clone achariso/gans-thesis repo into /content/code using git clone.
For more info see: https://medium.com/@purba0101/how-to-clone-private-github-repo-in-google-colab-using-ssh-77384cfef18f

In [ ]:
if is_first_run:
    !conda install git -y

import os

repo_root = '/workspace/code/gans-thesis'
ssh_root = '/root/.ssh'
!rm -rf "$repo_root"
!rm -rf "$ssh_root"
if not os.path.exists(repo_root) and not os.path.exists(f'{repo_root}/requirements.txt'):
    # Check that ssh keys exist
    assert os.path.exists(f'{drive_root}/GitHub Keys')
    id_rsa_abs_drive = f'{drive_root}/GitHub Keys/id_rsa'
    id_rsa_pub_abs_drive = f'{id_rsa_abs_drive}.pub'
    assert os.path.exists(id_rsa_abs_drive)
    assert os.path.exists(id_rsa_pub_abs_drive)
    # On first run: Add ssh key in repo
    if not os.path.exists(ssh_root) or True:
        # Transfer config file
        ssh_config_abs_drive = f'{drive_root}/GitHub Keys/config'
        assert os.path.exists(ssh_config_abs_drive)
        !mkdir -p "$ssh_root"
        !cp -f "$ssh_config_abs_drive" "$ssh_root/"
        # Add github.com to known hosts
        !ssh-keyscan -t rsa github.com >> "$ssh_root/known_hosts"
        # Test: !ssh -T git@github.com

    # Remove any previous attempts
    !rm -rf "$repo_root"
    !mkdir -p "$repo_root"
    # Clone repo
    !ssh -o StrictHostKeyChecking=no github.com
    !git clone git@github.com:achariso/gans-thesis.git "$repo_root" -o StrictHostKeyChecking=no
    src_root = f'{repo_root}/src'
    !rm -rf "$repo_root"/report

## 1.3) Install pip packages
All required files are stored in a requirements.txt files at the repository's root.
Use `pip install -r requirements.txt` from inside the dir to install required packages.

In [ ]:
%cd "$repo_root"
!pip install -r requirements.txt
!pip install kaggle --upgrade

# import os
# os.kill(os.getpid(), 9)

## 1.4) Add code/, */src/ to path
This is necessary in order to be able to run the modules.

In [ ]:
content_root_abs = f'{repo_root}'
src_root_abs = f'{repo_root}/src'
%env PYTHONPATH="/env/python:$content_root_abs:$src_root_abs"

# 2) Train StyleGAN model on DeepFashion's Fashion Image Synthesis Benchmark dataset
In this section we run the actual training loop for StyleGAN network. StyleGAN consists of a stylized generator and a
fairly naive discriminator architecture. Both however are progressively grown, starting from a resolution of 4x4 up to
the final resolution fo 128x128.

## Download dataset from Kaggle instead of Google Drive
This results in much - much faster download times.

In [ ]:
# Download Dataset from Kaggle
if not os.path.exists(f'/workspace/deepfashion-fisb-zip.zip'):
    !kaggle datasets download achariso/deepfashion-fisb-zip -p /workspace

### StyleGAN Training

Eventually, run the code!

In [ ]:
%cd src/modules/stylegan3
%run -i train.py \
    --outdir=/workspace/training-runs \
    --cfg=stylegan3-t \
    --data=/workspace/deepfashion-fisb-zip.zip \
    --cbase=16384 \
    --gpus=1 \
    --batch=32 \
    --gamma=0.5 \
    --batch-gpu=24 \
    --snap=10 \
    --metrics=fid50k